In [1]:
import stonpy

We need an uri, a user name and a password to connect to the database:

In [2]:
URI = "<uri>"
USER = "<username>"
PASSWORD = "<password>"

All operations are performed using a STON object that connects to the Neo4j database:

In [3]:
ston = stonpy.STON(URI, USER, PASSWORD)

The graph handled by the STON object is accessed using the `graph` property:

In [4]:
ston.graph

Graph('bolt://localhost:7687')

It is an instance of a [py2neo graph](https://py2neo.org/2021.1/workflow.html#graphservice-objects). It can be used to delete all data from the database, for example:

In [5]:
ston.graph.delete_all()

The STON object allows user to:

- add an SBGN map to the database
- retreive an SBGN map from the database
- delete an SBGN map from the database
- query the database, transforming the query result into valid SBGN maps or SBGN-ML files

In [6]:
sbgn_file = "insulin.sbgn"

We add the map to the database with the `create_map` method:

In [7]:
ston.create_map(sbgn_file, map_id="id1")

We then check that the database contains the map we've just created with the `has_map` method:

In [8]:
assert ston.has_map("id1") is True
assert ston.has_map(sbgn_map=sbgn_file) is True
assert ston.has_map(map_id="id1", sbgn_map=sbgn_file) is True
assert ston.has_map(map_id="id2", sbgn_map=sbgn_file) is False

We delete the map from the database with the `delete_map` method, and check that the map is no longer in the database:

In [9]:
ston.delete_map("id1")
assert ston.has_map("id1") is False

We retreive the map from the database with the `get_map` method, after adding it *de novo*:

In [10]:
ston.create_map(sbgn_file, map_id="id1")
sbgn_map = ston.get_map("id1")
sbgn_map # (<sbgn_map>, <id>)

(<libsbgnpy.libsbgn.map at 0x7f256d1b18d0>, 'id1')

We retreive the map and write it directly to an SBGN-ML file, using the `get_map_to_sbgn_file` method:

In [11]:
ston.get_map_to_sbgn_file("id1", "insulin_exported.sbgn")
with open("insulin_exported.sbgn") as f:
    print("".join(f.readlines()[:10]))

<?xml version="1.0" encoding="UTF-8"?>
<sbgn xmlns="http://sbgn.org/libsbgn/0.2">
    <map language="process description">
        <glyph class="macromolecule" id="glyph14" compartmentRef="glyph5">
            <label text="Grb2"/>
            <bbox w="108.0" h="60.0" x="721.0" y="575.0"/>
        </glyph>
        <glyph class="macromolecule" id="glyph15" compartmentRef="glyph5">
            <label text="c-Fos"/>
            <bbox w="108.0" h="60.0" x="1331.0" y="1070.0"/>



The database can be queried using the `graph` property and [py2neo](https://py2neo.org/) functionalities.
Here is a query that returns the each phosphorylation process node of the database:

In [12]:
# phosphorylation process
query = '''MATCH (process)-[:CONSUMES]->(reactant),
    (process)-[:PRODUCES]->(product),
    (reactant)-[:HAS_STATE_VARIABLE]-(reactant_sv),
    (product)-[:HAS_STATE_VARIABLE]-(product_sv)
    WHERE reactant.label = product.label
    AND reactant_sv.value IS NULL
    AND product_sv.value = "P"
    AND (product_sv.variable IS NULL
        AND reactant_sv.variable IS NULL
        AND product_sv.order = reactant_sv.order
        OR product_sv.variable = reactant_sv.variable)
    RETURN process;'''

cursor = ston.graph.run(query)
for record in cursor:
    print(record)

Node('GenericProcess', 'Glyph', 'Process', 'StoichiometricProcess', class='process', clone=False, id='glyph38', orientation='horizontal')
Node('GenericProcess', 'Glyph', 'Process', 'StoichiometricProcess', class='process', clone=False, id='glyph35', orientation='horizontal')
Node('GenericProcess', 'Glyph', 'Process', 'StoichiometricProcess', class='process', clone=False, id='glyph36', orientation='horizontal')


The STON object embeds methods that allow users to query the database and directly transform the result into valid SBGN maps or SBGN-ML files.
Rather than returning Neo4j nodes or relationships like in the example above, these methods complete each returned partial result (node(s) or relationship(s)) into one or more subgraphs that are then transformed into zero or more valid SBGN maps or SBGN-ML files.
There are two distinct methods: `query_to_map`, that returns SBGN maps from a query, and `query_to_sbgn_file`, that returns SBGN-ML files.
Each of these methods has a `merge_record` parameter, that changes the way the query results are merged before being completed.
When `merge_records` is set to `False` (the default), records are not merged before being completed; when it is set to `True`, records are merged before being completed.

We query the database for each phosporylation process node and return SBGN maps, with `merge_records` set to `False`:

In [13]:
sbgn_maps = ston.query_to_map(query, merge_records=False)
sbgn_maps # a generator

<generator object STON.query_to_map at 0x7f25978e7c30>

The query returns three distinct SBGN maps, one for each phosphroylation process found in the database:

In [14]:
for sbgn_map in sbgn_maps:
    print(sbgn_map)

(<libsbgnpy.libsbgn.map object at 0x7f256ce19e70>, 'id1')
(<libsbgnpy.libsbgn.map object at 0x7f256ce29ae0>, 'id1')
(<libsbgnpy.libsbgn.map object at 0x7f256ce18550>, 'id1')


We perform the same query, but with `merge_records` set to `True`:

In [15]:
sbgn_maps = ston.query_to_map(query, merge_records=True)
sbgn_maps # generator

<generator object STON.query_to_map at 0x7f25978e7450>

The query returns only one SBGN map, containing the three phosphorylation processes:

In [16]:
for sbgn_map in sbgn_maps:
    print(sbgn_map)

(<libsbgnpy.libsbgn.map object at 0x7f256d03b610>, 'id1')


Results of queries can be directly written to SBGN-ML files using the `query_to_sbgn_file` method:

In [17]:
sbgn_files = ston.query_to_sbgn_file(query, "result.sbgn", merge_records=False)
sbgn_files

['result_1.sbgn', 'result_2.sbgn', 'result_3.sbgn']

In [18]:
sbgn_files = ston.query_to_sbgn_file(query, "result.sbgn", merge_records=True)
sbgn_files

['result.sbgn']